In [1]:
cd Background

/home/kunjithapathams/Desktop/Data Analytics/OCR/Background


In [2]:
import random
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import os
from PIL import Image
from skimage import io
from skimage import transform as tf
import scipy.misc
import numpy as np
import cv2


In [53]:
alphanum = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']
fntSizes = [20,21,22,23,24,25,26,27,28,29,30]

def createsamples(text_pos,alpha,fnt,seq):
    bgimage = Image.new('RGBA', (40,40), (255, 255, 255, 255))        
    draw  = ImageDraw.Draw(bgimage)
    draw.text(text_pos,alpha,font=fnt,fill=(0,0,0))
    bgimage.save('samples/'+alpha+'_'+str(seq)+'.png')
        
for alpha in alphanum:
    #print(str(seq))
    seq = 1
    for fntSize in fntSizes:
        for i in range (50):
            text_Pos = (10,10)                
            fnt = ImageFont.truetype('arialbd.ttf', fntSize)
            createsamples(text_Pos,alpha,fnt,seq)
            seq = seq +1
                
print ('samples created....')
           


samples created....


In [6]:
def noisy(noise_typ,image):
    
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

for filename in os.listdir('samplesBW/'):        
    image = cv2.imread('samplesBW/'+filename)
    #image = noisy('gauss',image)
    #image = noisy('poisson',image)
    image = cv2.blur(image,(2,2))
    ##print(image)
    cv2.imwrite('samplesBW/'+filename,image)
    #image = noisy(image)
    #print(filename)
    #blendImage(image,'samplesBW/'+filename)
    #cv2.imwrite('samplesBW/'+filename,image)

print('noice induced')



noice induced


In [7]:
print('start reshaping')
alphanum = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']
reshaped = np.empty([19800,1600])
Y = np.chararray([(len(alphanum))*550,1]) 
j=0
for alpha in alphanum:
    print(alpha)
    for i in range(1,550):
        filename = 'samplesBW/'+alpha+'_'+str(i)+'.png'
        image = io.imread(filename,'0')
        reshaped[j] = image.reshape(1,1600)
        #reshaped = np.concatenate([reshaped,image.reshape(1,2500)])
        Y[j] = alpha
        j = j+1

print('Reshaping Complete')
print('Starting to segregate Training and Validation sets')
Y = np.ravel(Y)
from sklearn.model_selection import train_test_split
X_Train,X_Test,Y_Train,Y_Test = train_test_split(reshaped,Y)


print('Segregation Completed')

start reshaping
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z
0
1
2
3
4
5
6
7
8
9
Reshaping Complete
Starting to segregate Training and Validation sets
Segregation Completed


In [8]:
print('starting to feature scaling')

#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
# Fit only to the training data
#scaler.fit(X_Train)
#X_Train = scaler.transform(X_Train)
#X_Test = scaler.transform(X_Test)

print('Feature Scaling Completed')
print('Start training')
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30),alpha=0.001)
mlp.fit(X_Train,Y_Train)


print('Training done...')
print('Now start predicting for the validation set')
predictions = mlp.predict(X_Test)

print('Predictions done')
print('evaluate the results')
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(Y_Test,predictions))

starting to feature scaling
Feature Scaling Completed
Start training
Training done...
Now start predicting for the validation set
Predictions done
evaluate the results
             precision    recall  f1-score   support

        b''       0.40      1.00      0.57         4
    b'\x0f'       0.00      0.00      0.00         1
       b'0'       1.00      0.65      0.79       137
       b'1'       0.98      0.99      0.98       122
       b'2'       1.00      0.89      0.94       137
       b'3'       0.99      1.00      0.99       160
       b'4'       0.99      0.99      0.99       132
       b'5'       0.90      0.89      0.89       146
       b'6'       0.75      0.72      0.73       136
       b'7'       0.88      0.89      0.89       131
       b'8'       0.83      0.91      0.87       143
       b'9'       0.99      1.00      1.00       140
       b'A'       0.75      0.77      0.76       130
       b'B'       0.84      1.00      0.91       124
       b'C'       0.89      1.00    

/home/kunjithapathams/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
for filename in os.listdir('Test/'):  
    #BGImage = Image.open('defaultBG.png')
    #img = Image.open('Test/'+filename)    
    #BGImage.paste(img, (15,15))    
    #BGImage.save('TestBW/'+filename)
    
    image = cv2.imread('Test/'+filename)
    gray_masked = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    (thresh, im_bw) = cv2.threshold(gray_masked, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    cv2.imwrite('TestBW/'+filename,im_bw)
    #blendImage(image,'TestBW/'+filename)
    
print('Test images created...')

Test images created...


In [10]:
#Paste the BW image onto a standard 50X50 white background
for filename in os.listdir('TestBW/'):
    img = Image.open('TestBW/'+filename)
    bgimage = Image.new('RGBA', (40,40), (255, 255, 255, 255))        
    pos = (10,10)
    bgimage.paste(img,pos)
    bgimage.save('TestBW/'+filename)
    
    image = cv2.imread('TestBW/'+filename)
    image = cv2.blur(image,(2,2))
    cv2.imwrite('TestBW/'+filename,image)
    
print('Test images created...')




Test images created...


In [11]:
print('starting to reshape the test images')
pannumber = ['A','W','S','P','K','5','9','6','7','E']
reshaped1 = np.empty([10,1600])
Y1=np.chararray([len(pannumber),1])
j=0
for alpha in pannumber:
    #for i in range (1,3):
        filename = 'TestBW/'+alpha+'_1.PNG'
        #print(filename)
        image = io.imread(filename,'1')    
        #print(image)
        reshaped1[j] = image.reshape(1,1600)
        Y1[j] = alpha
        j = j+1

Y1 = np.ravel(Y1)

print('reshaping done...')
print('start predicting')
#X1_Test = scaler.transform(reshaped1)
predictions = mlp.predict(reshaped1)
print(predictions)
print(classification_report(Y1,predictions))

starting to reshape the test images
reshaping done...
start predicting
[b'A' b'1' b'M' b'A' b'K' b'6' b'4' b'R' b'7' b'F']
             precision    recall  f1-score   support

       b'1'       0.00      0.00      0.00         0
       b'4'       0.00      0.00      0.00         0
       b'5'       0.00      0.00      0.00         1
       b'6'       0.00      0.00      0.00         1
       b'7'       1.00      1.00      1.00         1
       b'9'       0.00      0.00      0.00         1
       b'A'       0.50      1.00      0.67         1
       b'E'       0.00      0.00      0.00         1
       b'F'       0.00      0.00      0.00         0
       b'K'       1.00      1.00      1.00         1
       b'M'       0.00      0.00      0.00         0
       b'P'       0.00      0.00      0.00         1
       b'R'       0.00      0.00      0.00         0
       b'S'       0.00      0.00      0.00         1
       b'W'       0.00      0.00      0.00         1

avg / total       0.25     

/home/kunjithapathams/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/kunjithapathams/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [37]:
print(predictions)

[b'A' b'A' b'J' b'J' b'R' b'R' b'1' b'1' b'M' b'M' b'0' b'0' b'7' b'7' b'0'
 b'0' b'T' b'T' b'F' b'F']
